## LSTM Models

- Single Layer LSTM
- Multiple Layers LSTM
- LSTM with Attention

## Model Structures

In [2]:
from tensorflow.keras.utils import plot_model
from data import *
from lstm_model import *

model_types = ['single_layer_lstm']#, 'multi_layer_lstm', 'lstm_attention', 'attention_lstm']
_, ds_rolling = get_dataset_trends()
for model_type in model_types:
    configs = {
        'dataset': ds_rolling,
        'model_path': f"./models/{model_type}_trends",
        'model_type': model_type,
    }
    model = LSTMModel(**configs)
    model.read_model()
    plot_model(model.model, to_file=f'{model_type}.png')

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## Activation Experiment

Test on 4 activation functions in LSTM, including:

- sigmoid
- tanh
- relu
- linear

In [ ]:
import seaborn as sns
import pickle as pkl
from matplotlib import pyplot as plt

from data import *
from lstm_model import *

# Set plotting style
sns.set_style('whitegrid')
sns.set_palette('Set2')


In [ ]:
# Get confirmed cases dataset
dataset, dataset_rolling = get_dataset_confirmed()
# Print loss of different activations
model_type = 'single_layer_lstm'
activations = ['sigmoid', 'tanh', 'linear', 'relu']
for activation in activations:
    configs = {
        'dataset': dataset_rolling,
        'model_path': f"./models/{model_type}_confirmed_{activation}",
        'model_type': model_type,
        'activation': activation,
    }
    model = LSTMModel(**configs)
    model.read_model()
    # model.plot(f'Confirmed Cases Forcasting with Activation: {activation}')
    model.evaluate()

In [ ]:
import pickle

activations = [ 'tanh', 'linear', 'relu']
# Print loss of different activations
for activation in activations:
    history_path = f"./models/{model_type}_confirmed_{activation}/history.pkl"
    history_file = open(history_path, 'rb')
    history = pickle.load(history_file)

    plt.figure(figsize=(10, 8))
    plt.plot(history['loss'])
    plt.title(f'Model Loss of {activation}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    # plt.legend(['Loss'], loc='upper left')
    plt.show()

## Forcasting Google Trends